In [1]:
import pandas as pd
import glob
from datetime import datetime

In [4]:
q = pd.read_csv("monthlies/2018_03.csv")
q["status"].unique()

array(['departed', 'estimated', 'cancelled'], dtype=object)

In [2]:
# get all .csv files in a list
files = glob.glob("monthlies/*.csv")

trains = pd.concat([pd.read_csv(f) for f in files], ignore_index = True)

In [5]:
trains = trains[trains["type"]=="NJ Transit"]
trains = trains[trains["status"].isin(["departed","estimated"])]
trains = trains.dropna()

In [6]:
trains = trains.rename(columns={"scheduled_time":"scheduled_datetime","actual_time":"actual_datetime"})

In [7]:
trains[["scheduled_date","scheduled_time"]] = trains["scheduled_datetime"].str.split(" ",expand=True,)
trains[["actual_date","actual_time"]] = trains["actual_datetime"].str.split(" ",expand=True,)

In [8]:
trains = trains.drop(columns=["scheduled_datetime","actual_datetime","date"])

In [9]:
trains = trains.reset_index()

In [21]:
inbound = "New York Penn Station"

q = trains[trains["to"]==inbound]
# first stop is just origin station to origin station
q = q[q["stop_sequence"]>1]

In [22]:
from_ny = trains[(trains["from"]=="New York Penn Station") & (trains["stop_sequence"]>1)]
from_ids = from_ny["train_id"]

from_trains = trains[(trains["train_id"].isin(from_ids)) & (trains["stop_sequence"]>1)]
from_trains = from_trains.reset_index()

In [23]:
# w = pd.DataFrame(columns=trains.columns)

#start_train_id = from_trains["train_id"].iloc[0]
#location = 0
#for index, row in from_trains.iterrows():
#    if row["train_id"]==start_train_id:
#        pass
#    else:
#        w.loc[location] = from_trains.iloc[index-1]
#        location = location + 1
#        start_train_id = row["train_id"]

In [24]:
dow = ("Mon","Tue","Wed","Thu","Fri","Sat","Sun")

qweek_day = []
for index, row in q.iterrows():
    thisdate = datetime.strptime(row["scheduled_date"], "%Y-%m-%d")
    qweek_day.append(dow[thisdate.weekday()])
    
#wweek_day = []
#for index, row in w.iterrows():
#    thisdate = datetime.strptime(row["scheduled_date"], "%Y-%m-%d")
#    wweek_day.append(dow[thisdate.weekday()])

In [25]:
q["week_day"] = qweek_day
#w["week_day"] = wweek_day

In [26]:
lines = q["line"].unique()
len(lines)

6

In [27]:
q = q.reset_index()

In [29]:
q.head()

,level_0,index,train_id,stop_sequence,from,from_id,to,to_id,delay_minutes,status,line,type,scheduled_date,scheduled_time,actual_date,actual_time,week_day
0,65,82,7812,14.0,Secaucus Upper Lvl,38187.0,New York Penn Station,105.0,0.0,estimated,Northeast Corrdr,NJ Transit,2019-01-01,08:32:00,2019-01-01,08:31:00,Tue
1,79,130,7824,14.0,Secaucus Upper Lvl,38187.0,New York Penn Station,105.0,0.0,estimated,Northeast Corrdr,NJ Transit,2019-01-01,11:34:00,2019-01-01,11:33:00,Tue
2,121,172,7848,14.0,Secaucus Upper Lvl,38187.0,New York Penn Station,105.0,11.0,estimated,Northeast Corrdr,NJ Transit,2019-01-01,17:36:00,2019-01-01,17:47:00,Tue
3,183,300,7846,11.0,Secaucus Upper Lvl,38187.0,New York Penn Station,105.0,2.0,estimated,Northeast Corrdr,NJ Transit,2019-01-01,16:46:00,2019-01-01,16:48:00,Tue
4,221,343,7822,11.0,Secaucus Upper Lvl,38187.0,New York Penn Station,105.0,0.0,estimated,Northeast Corrdr,NJ Transit,2019-01-01,10:43:00,2019-01-01,10:42:00,Tue


In [30]:
for linename in lines:
    qtosave = q[q["line"]==linename]
    qtosave.to_csv(r"bylines/{}-TO.csv".format(linename),index=False)
    
    #wtosave = q[q["line"]==linename]
    #wtosave.to_csv(r"bylines/{}-FROM.csv".format(linename),index=False)